In [2]:
import pandas as pd
from nltk import FreqDist
import pickle
import numpy as np
from gensim.models import Word2Vec

In [3]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [4]:
LAGT = pd.read_json("../data/large_data/LAGT_metadata.json")

In [5]:
LAGT.head(5)

,author_id,doc_id,GLAUx?,filename,subcorpus,author,title,date_avr,provenience,tlg_epithet,n_sentences,wordcount
0,tlg0001,tlg0001.tlg001,True,tlg0001.tlg001.perseus-grc2.xml,classical,Apollonius Rhodius,Argonautica,-2.5,pagan,Epici/-ae,3252,38822
1,tlg0003,tlg0003.tlg001,True,tlg0003.tlg001.perseus-grc2.xml,classical,Thucydides,The Peloponnesian War,-4.5,pagan,Historici/-ae,6068,150118
2,tlg0004,tlg0004.tlg001,False,tlg0004.tlg001.perseus-grc1.xml,roman,Diogenes Laertius,Lives of Eminent Philosophers,2.5,pagan,Biographi,10245,110763
3,tlg0005,tlg0005.tlg001,True,tlg0005.tlg001.perseus-grc1.xml,classical,Theocritus,Idylls,-3.0,pagan,Bucolici,1982,19200
4,tlg0005,tlg0005.tlg002,True,tlg0005.tlg002.perseus-grc1.xml,classical,Theocritus,Epigrams,-3.0,pagan,Bucolici,152,1734


In [6]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

religion_final = ["θεός", "Ζεύς", "εὐσεβής", 'ἱερός']
morality_final = ["ἀγαθός", "ἀρετή", "δίκαιος", "τιμή"]

# Read subcorpora from ngrams - developing a memory-friendly reader

In [7]:
ids_lines = pickle.load(open("../data/ids_lines.pickle", "rb"))

In [8]:
list(ids_lines.items())[:5]

[('tlg0001.tlg001', (0, 41193)),
 ('tlg0003.tlg001', (41193, 184616)),
 ('tlg0004.tlg001', (184616, 280036)),
 ('tlg0005.tlg001', (280036, 300693)),
 ('tlg0005.tlg002', (300693, 302641))]

In [9]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="archaic"]["doc_id"].tolist()
len(subcorpus_ids)

26

In [10]:
lines_list = [ ]
for id in subcorpus_ids:
    lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
ngrams_n = len(lines_list)
ngrams_n

339423

In [11]:
sample_size=1000000
if sample_size != None:
    sample_inx =np.random.randint(0, ngrams_n, sample_size)
else:
    sample_inx =[n for n in range(0, ngrams_n)]

In [12]:
sample_lines = np.array(lines_list)[sample_inx]
len(sample_lines)

1000000

In [13]:
sample_lines[:5]

array([8984361, 2010234, 1946059, 2032547, 8984901])

In [14]:
test_ngrams = []
with open("../data/large_data/corpus_ngrams_bydocid.txt", "r") as f:
    lines = f.readlines()
    for li in sample_lines:
        line = lines[li]# for line index in subcorpus lineindexes
        if bool(line.split()):
            test_ngrams.append(line.split())

In [15]:
len(test_ngrams)

999605

# Sample corpora with memory-friendly reader

In [16]:
class NgramCorpusSample(object):
    def __init__(self, ids_list, ids_lines, fname, sample_size=None, sample_seed=1, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.sample_size = sample_size
        self.ids_lines = ids_lines
        lines_list = [ ]
        for id in ids_list:
            lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
        ngrams_n = len(lines_list)
        if sample_size != None:
            np.random.seed(sample_seed)
            sample_inx =np.random.randint(0, ngrams_n, sample_size)
        else:
            sample_inx =[n for n in range(0, ngrams_n)]
        sample_lines = np.array(lines_list)[sample_inx]
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for li in self.sample_lines:
                line = lines[li] # for line index in subcorpus lineindexes
                if bool(line.split()):
                    if (self.bow) & (self.dct != None):
                        yield self.dct.doc2bow(line.split())
                    else:
                        yield line.split()

In [17]:
# test with classical

In [18]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="roman"]["doc_id"].tolist()
len(subcorpus_ids)

596

In [19]:
subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000)

In [20]:
len(subcorpus_ngrams)

1000000

In [21]:
len([doc for doc in subcorpus_ngrams])

980208

In [22]:
FreqDist([w for ng in subcorpus_ngrams for w in ng]).most_common()[:50]

[('εἰμί', 86416),
 ('αὐτός', 68360),
 ('οὗτος', 55211),
 ('λέγω', 37546),
 ('γίγνομαι', 31525),
 ('τις', 30334),
 ('ἔχω', 29557),
 ('πολύς', 28087),
 ('πᾶς', 22650),
 ('ἄλλος', 21668),
 ('φημί', 17161),
 ('ποιέω', 15230),
 ('λόγος', 14224),
 ('ἐκεῖνος', 11745),
 ('τοιοῦτος', 11694),
 ('μέγας', 11540),
 ('πρῶτος', 10766),
 ('οὐδείς', 9887),
 ('ἕτερος', 9696),
 ('πόλις', 9384),
 ('σῶμα', 8978),
 ('ἅπας', 8360),
 ('ἀγαθός', 8282),
 ('φύσις', 8149),
 ('δύναμις', 8130),
 ('μέρος', 7951),
 ('ἄνθρωπος', 7917),
 ('εἷς', 7831),
 ('ἀρχή', 7814),
 ('λαμβάνω', 7582),
 ('ἕκαστος', 7256),
 ('δοκέω', 7100),
 ('ὁράω', 6997),
 ('δέω', 6949),
 ('μόνος', 6823),
 ('ἀνήρ', 6793),
 ('καλέω', 6710),
 ('δύναμαι', 6256),
 ('ὑπάρχω', 6136),
 ('χρόνος', 5940),
 ('θεός', 5916),
 ('χράω', 5609),
 ('δίδωμι', 5491),
 ('ὅλος', 4946),
 ('ἡμέρα', 4905),
 ('μηδείς', 4883),
 ('γράφω', 4851),
 ('γένος', 4826),
 ('τὶς', 4718),
 ('τίς', 4716)]

In [23]:
with open("../data/large_data/subcorpora_vocabs.pickle", "rb") as f:
    subcorpora_vocabs = pickle.load(f)

In [24]:
subcorpora_vocabs["christian"]["word_freqs"][:100]

[('εἰμί', 18108),
 ('λέγω', 15873),
 ('θεός', 13922),
 ('οὗτος', 11814),
 ('αὐτός', 11282),
 ('πᾶς', 7431),
 ('γίγνομαι', 7316),
 ('λόγος', 6836),
 ('ἄνθρωπος', 5723),
 ('ἔχω', 5375),
 ('κύριος', 5300),
 ('φημί', 5294),
 ('ποιέω', 4416),
 ('πολύς', 4229),
 ('Ἰησοῦς', 4155),
 ('ὁράω', 3320),
 ('πατήρ', 3158),
 ('Χριστός', 3087),
 ('ψυχή', 3037),
 ('γῆ', 2859),
 ('ἄλλος', 2826),
 ('υἱός', 2753),
 ('δύναμαι', 2654),
 ('πνεῦμα', 2587),
 ('τις', 2552),
 ('ἔρχομαι', 2260),
 ('ἀκούω', 2175),
 ('σῶμα', 2130),
 ('μόνος', 2087),
 ('τοιοῦτος', 2069),
 ('δίδωμι', 2049),
 ('ἀγαθός', 2034),
 ('δύναμις', 1981),
 ('οὐρανός', 1881),
 ('λαμβάνω', 1878),
 ('ἐκεῖνος', 1854),
 ('ὄνομα', 1846),
 ('οἶδα', 1776),
 ('τὶς', 1770),
 ('οὐδείς', 1765),
 ('προφήτης', 1660),
 ('ἅγιος', 1653),
 ('μέγας', 1641),
 ('πιστεύω', 1631),
 ('ἡμέρα', 1628),
 ('κόσμος', 1626),
 ('νόμος', 1593),
 ('καλέω', 1481),
 ('ἀνήρ', 1417),
 ('ἔργον', 1417),
 ('βούλομαι', 1400),
 ('πρῶτος', 1390),
 ('ἕτερος', 1387),
 ('εἷς', 1370),
 ('ὅλο

In [25]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="christian"]["doc_id"].tolist()
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:100])

{'εἰμί': 68779,
 'θεός': 54401,
 'λέγω': 53299,
 'αὐτός': 44000,
 'οὗτος': 39635,
 'πᾶς': 29215,
 'γίγνομαι': 27883,
 'λόγος': 26969,
 'ἄνθρωπος': 22561,
 'ἔχω': 20629,
 'κύριος': 19568,
 'φημί': 17691,
 'ποιέω': 16660,
 'πολύς': 16373,
 'Ἰησοῦς': 16227,
 'πατήρ': 12170,
 'Χριστός': 12010,
 'ψυχή': 11777,
 'ὁράω': 11503,
 'ἄλλος': 10979,
 'υἱός': 10971,
 'τις': 10790,
 'γῆ': 10742,
 'πνεῦμα': 10233,
 'δύναμαι': 9945,
 'σῶμα': 8386,
 'μόνος': 8260,
 'ἔρχομαι': 8161,
 'δύναμις': 8053,
 'τοιοῦτος': 7643,
 'δίδωμι': 7637,
 'ἀγαθός': 7582,
 'ἀκούω': 7534,
 'ὄνομα': 7222,
 'οὐρανός': 7217,
 'λαμβάνω': 7091,
 'ἐκεῖνος': 6962,
 'ἅγιος': 6598,
 'μέγας': 6584,
 'τὶς': 6472,
 'προφήτης': 6457,
 'κόσμος': 6421,
 'οὐδείς': 6405,
 'ἡμέρα': 6269,
 'νόμος': 6201,
 'οἶδα': 6183,
 'πιστεύω': 6009,
 'καλέω': 5594,
 'ἀνήρ': 5586,
 'ἔργον': 5528,
 'εἷς': 5356,
 'ὅλος': 5325,
 'πρῶτος': 5189,
 'ἕτερος': 5164,
 'βούλομαι': 5107,
 'ἀρχή': 5094,
 'ἀλήθεια': 5040,
 'ἔθνος': 5026,
 'φύσις': 5013,
 'Ἰουδαῖος': 49

In [26]:
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)

In [27]:
freqs_comparison = pd.DataFrame(
    [dict(subcorpora_vocabs["roman"]["word_freqs"][:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed2 for w in ng]).most_common()[:5000])]
).T

In [28]:
freqs_comparison.dropna().corr()

,0,1,2
0,1.000000,0.847227,0.846015
1,0.847227,1.000000,0.999908
2,0.846015,0.999908,1.000000


In [29]:
len(freqs_comparison)

7185

In [30]:
freqs_comparison[:4000].corr(method="spearman")

,0,1,2
0,1.000000,0.592499,0.592932
1,0.592499,1.000000,0.992367
2,0.592932,0.992367,1.000000


In [31]:
np.log2(freqs_comparison).corr(method="spearman")

,0,1,2
0,1.000000,0.590499,0.592962
1,0.590499,1.000000,0.982044
2,0.592962,0.982044,1.000000


In [32]:
# let's build two models based on the same subcorpus but sampled with different seed

In [33]:
%%time
sub = "christian"

ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))

subcorpus_ids = LAGT[LAGT["subcorpus"]==sub]["doc_id"].tolist()
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=2000000, sample_seed=1)
model_seed1 = Word2Vec(vector_size=300, window=5, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed1.train(subcorpus_ngrams_seed1, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
vectors_seed1 = model_seed1.wv

subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=2000000, sample_seed=2)
model_seed2 = Word2Vec(vector_size=300, window=5, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed2.train(subcorpus_ngrams_seed2, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
vectors_seed2 = model_seed2.wv

CPU times: user 3min 19s, sys: 2min 7s, total: 5min 27s
Wall time: 4min 38s


In [36]:
ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))

for sub in subcorpora:
    subcorpus_ids = LAGT[LAGT["subcorpus"]==sub]["doc_id"].tolist()
    #do the stuff above
    print("generating model for the {} subcorpus...".format(sub))
    subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000, sample_seed=1)
    model_seed1 = Word2Vec(vector_size=150, window=5, negative=5, ns_exponent=1, sg=0, epochs=10, workers=8)
    model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed1.train(subcorpus_ngrams_seed1, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
    vectors_seed1 = model_seed1.wv
    vectors_seed1.save("../data/large_data/vectors_{}_seed1.kv".format(sub))
    subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000, sample_seed=2)
    model_seed2 = Word2Vec(vector_size=150, window=5, negative=5, ns_exponent=1, sg=0, epochs=10, workers=8)
    model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed2.train(subcorpus_ngrams_seed1, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
    vectors_seed2 = model_seed2.wv
    vectors_seed2.save("../data/large_data/vectors_{}_seed2.kv".format(sub))
    print("{} model ready".format(sub))


generating model for the archaic subcorpus...
archaic model ready
generating model for the classical subcorpus...
classical model ready
generating model for the roman subcorpus...
roman model ready
generating model for the christian subcorpus...
christian model ready
generating model for the jewish subcorpus...
jewish model ready


In [37]:
ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))

for sub in subcorpora:
    subcorpus_ids = LAGT[LAGT["subcorpus"]==sub]["doc_id"].tolist()
    #do the stuff above
    print("generating model for the {} subcorpus...".format(sub))
    subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=None, sample_seed=1)
    model_seed1 = Word2Vec(vector_size=150, window=5, negative=5, ns_exponent=1, sg=0, epochs=10, workers=8)
    model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed1.train(subcorpus_ngrams_seed1, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
    vectors_seed1 = model_seed1.wv
    vectors_seed1.save("../data/large_data/vectors_{}_seed1_notsampled.kv".format(sub))
    subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=None, sample_seed=2)
    model_seed2 = Word2Vec(vector_size=150, window=5, negative=5, ns_exponent=1, sg=0, epochs=10, workers=8)
    model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed2.train(subcorpus_ngrams_seed1, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
    vectors_seed2 = model_seed2.wv
    vectors_seed2.save("../data/large_data/vectors_{}_seed2_notsampled.kv".format(sub))
    print("{} model ready".format(sub))

generating model for the archaic subcorpus...
archaic model ready
generating model for the classical subcorpus...
classical model ready
generating model for the roman subcorpus...
roman model ready
generating model for the christian subcorpus...
christian model ready
generating model for the jewish subcorpus...
jewish model ready
